<a href="https://colab.research.google.com/github/Sajithpemarathna/Energy-Consumption-Data-in-Germany-1995-2020-/blob/main/SQL_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries for SQLite and Pandas
!pip install pandas xlrd openpyxl


In [ ]:
import pandas as pd

# Load the cleaned Excel files
branches_df = pd.read_excel('Energy Consumption of Homogeneous Branches.xlsx')
carriers_df = pd.read_excel('Use of Energy of Energy Carriers.xlsx')

# Display the first few rows to verify the data
print("Branches Data:")
print(branches_df.head())

print("\nCarriers Data:")
print(carriers_df.head())


Branches Data:
  Branch Code                               Homogeneous Branches    1995  \
0    CPA08-01   Products of agric., hunting and related services  194596   
1    CPA08-02  Products of forestry, logging and related serv...    7264   
2    CPA08-03  Fish,aquaculture prod.,support services to fis...    2882   
3    CPA08-05                                   Coal and lignite   33934   
4    CPA08-06                    Crude petroleum and natural gas   20304   

     1996    1997    1998    1999    2000    2001    2002  ...    2011  \
0  245008  221128  215325  201556  187373  201202  194596  ...  156271   
1    7941    7620    7254    8316    8657    7240    7264  ...    6382   
2    4155    3350    3720    3137    2981    2878    2882  ...    2327   
3   76847   71506   60658   52097   36337   34736   33934  ...   39280   
4   26462   24269   25027   24370   23675   22586   20304  ...   11555   

     2012    2013    2014    2015    2016    2017    2018    2019    2020  
0  1951

In [ ]:
import sqlite3

# Create an SQLite database in memory
conn = sqlite3.connect(':memory:')

# Write DataFrames to SQL tables
branches_df.to_sql('Energy_Consumption_Branches', conn, index=False, if_exists='replace')
carriers_df.to_sql('Energy_Use_Carriers', conn, index=False, if_exists='replace')

# Verify tables are loaded
print("Tables in the database:")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)


Tables in the database:
                          name
0  Energy_Consumption_Branches
1          Energy_Use_Carriers


In [ ]:
query1 = """
SELECT Year, SUM(Energy_Consumption) AS Total_Consumption
FROM Energy_Consumption_Branches
GROUP BY Year
ORDER BY Year ASC;
"""
result1 = pd.read_sql_query(query1, conn)
print(result1)


DatabaseError: Execution failed on sql '
SELECT Year, SUM(Energy_Consumption) AS Total_Consumption
FROM Energy_Consumption_Branches
GROUP BY Year
ORDER BY Year ASC;
': no such column: Year

In [ ]:
query_check = """
PRAGMA table_info(Energy_Consumption_Branches);
"""
table_info = pd.read_sql_query(query_check, conn)
print(table_info)


    cid                  name     type  notnull dflt_value  pk
0     0           Branch Code     TEXT        0       None   0
1     1  Homogeneous Branches     TEXT        0       None   0
2     2                  1995  INTEGER        0       None   0
3     3                  1996  INTEGER        0       None   0
4     4                  1997  INTEGER        0       None   0
5     5                  1998  INTEGER        0       None   0
6     6                  1999  INTEGER        0       None   0
7     7                  2000  INTEGER        0       None   0
8     8                  2001  INTEGER        0       None   0
9     9                  2002  INTEGER        0       None   0
10   10                  2003  INTEGER        0       None   0
11   11                  2004  INTEGER        0       None   0
12   12                  2005  INTEGER        0       None   0
13   13                  2006  INTEGER        0       None   0
14   14                  2007  INTEGER        0       N

In [ ]:
# Melt the dataframe to reshape it into a long format
branches_long = branches_df.melt(id_vars=["Branch Code", "Homogeneous Branches"],
                                  var_name="Year",
                                  value_name="Energy_Consumption")

# Convert the Year column to an integer for consistency
branches_long["Year"] = branches_long["Year"].astype(int)

# Display the reshaped dataframe
print(branches_long.head())


  Branch Code                               Homogeneous Branches  Year  \
0    CPA08-01   Products of agric., hunting and related services  1995   
1    CPA08-02  Products of forestry, logging and related serv...  1995   
2    CPA08-03  Fish,aquaculture prod.,support services to fis...  1995   
3    CPA08-05                                   Coal and lignite  1995   
4    CPA08-06                    Crude petroleum and natural gas  1995   

   Energy_Consumption  
0              194596  
1                7264  
2                2882  
3               33934  
4               20304  


In [ ]:
# Reload the reshaped data into the database
branches_long.to_sql('Energy_Consumption_Branches', conn, index=False, if_exists='replace')

# Verify the new structure
query_check = """
PRAGMA table_info(Energy_Consumption_Branches);
"""
table_info = pd.read_sql_query(query_check, conn)
print(table_info)


   cid                  name     type  notnull dflt_value  pk
0    0           Branch Code     TEXT        0       None   0
1    1  Homogeneous Branches     TEXT        0       None   0
2    2                  Year  INTEGER        0       None   0
3    3    Energy_Consumption  INTEGER        0       None   0


In [ ]:
query1 = """
SELECT Year, SUM(Energy_Consumption) AS Total_Consumption
FROM Energy_Consumption_Branches
GROUP BY Year
ORDER BY Year ASC;
"""
result1 = pd.read_sql_query(query1, conn)
print(result1)


    Year  Total_Consumption
0   1995           14357268
1   1996           14683191
2   1997           14513045
3   1998           14405142
4   1999           14236759
5   2000           14290357
6   2001           14590137
7   2002           14357268
8   2003           14521745
9   2004           14515959
10  2005           14738830
11  2006           14969148
12  2007           14458573
13  2008           14751605
14  2009           13850025
15  2010           14717813
16  2011           14026360
17  2012           13831179
18  2013           14118917
19  2014           13405145
20  2015           13669827
21  2016           13679308
22  2017           13678181
23  2018           13399931
24  2019           13022128
25  2020           11977459


In [ ]:
SELECT Carrier_Type, Year, SUM(Energy_Usage) AS Total_Consumption
FROM Energy_Use_Carriers
WHERE Carrier_Type != 'Total'
GROUP BY Carrier_Type, Year
ORDER BY Carrier_Type, Year;


SyntaxError: invalid syntax (<ipython-input-9-307cd85b32df>, line 1)

In [ ]:
query_check = """
PRAGMA table_info(Energy_Use_Carriers);
"""
table_info = pd.read_sql_query(query_check, conn)
print(table_info)


    cid                  name  type  notnull dflt_value  pk
0     0           Branch Code  TEXT        0       None   0
1     1  Homogeneous Branches  TEXT        0       None   0
2     2       Energy Carriers  TEXT        0       None   0
3     3                  1995  REAL        0       None   0
4     4                  1996  REAL        0       None   0
5     5                  1997  REAL        0       None   0
6     6                  1998  REAL        0       None   0
7     7                  1999  REAL        0       None   0
8     8                  2000  REAL        0       None   0
9     9                  2001  REAL        0       None   0
10   10                  2002  REAL        0       None   0
11   11                  2003  REAL        0       None   0
12   12                  2004  REAL        0       None   0
13   13                  2005  REAL        0       None   0
14   14                  2006  REAL        0       None   0
15   15                  2007  REAL     

In [ ]:
# Reshape the data into a long format
carriers_long = carriers_df.melt(id_vars=["Branch Code", "Homogeneous Branches", "Energy Carriers"],
                                 var_name="Year",
                                 value_name="Energy_Usage")

# Convert the Year column to integer for consistency
carriers_long["Year"] = carriers_long["Year"].astype(int)

# Display the reshaped DataFrame
print(carriers_long.head())

# Save the reshaped DataFrame into SQLite
carriers_long.to_sql('Energy_Use_Carriers_Long', conn, index=False, if_exists='replace')


  Branch Code                              Homogeneous Branches  \
0    CPA08-01  Products of agric., hunting and related services   
1         NaN                                               NaN   
2         NaN                                               NaN   
3         NaN                                               NaN   
4         NaN                                               NaN   

                                Energy Carriers  Year  Energy_Usage  
0              Hard coal and hard coal products  1995         888.0  
1  Brown coal (lignite) and brown coal products  1995        1477.0  
2                                     Crude oil  1995           0.0  
3                                        Petrol  1995        4580.0  
4                                  Diesel fuels  1995       94156.0  


16640

In [ ]:
query_total_carriers = """
SELECT "Energy Carriers", Year, SUM(Energy_Usage) AS Total_Consumption
FROM Energy_Use_Carriers_Long
WHERE "Energy Carriers" != 'Total'
GROUP BY "Energy Carriers", Year
ORDER BY "Energy Carriers", Year;
"""
result_carriers = pd.read_sql_query(query_total_carriers, conn)
print(result_carriers)


           Energy Carriers  Year  Total_Consumption
0    Aviation turbine fuel  1995           696728.0
1    Aviation turbine fuel  1996           719250.0
2    Aviation turbine fuel  1997           760899.0
3    Aviation turbine fuel  1998           779452.0
4    Aviation turbine fuel  1999           831340.0
..                     ...   ...                ...
307     Renewable energies  2016          3503515.0
308     Renewable energies  2017          3746369.0
309     Renewable energies  2018          3746719.0
310     Renewable energies  2019          3937624.0
311     Renewable energies  2020          4047305.0

[312 rows x 3 columns]


In [ ]:
query_total_year = """
SELECT Year, SUM(Energy_Usage) AS Total_Consumption
FROM Energy_Use_Carriers_Long
WHERE "Energy Carriers" = 'Total'
GROUP BY Year
ORDER BY Year;
"""
result_total_year = pd.read_sql_query(query_total_year, conn)
print(result_total_year)


    Year  Total_Consumption
0   1995         42162734.0
1   1996         42902541.0
2   1997         42401596.0
3   1998         42837106.0
4   1999         42264119.0
5   2000         43055046.0
6   2001         43032040.0
7   2002         42616001.0
8   2003         44041388.0
9   2004         45069426.0
10  2005         46284572.0
11  2006         46822644.0
12  2007         45814327.0
13  2008         45698196.0
14  2009         42497279.0
15  2010         43903839.0
16  2011         42565582.0
17  2012         42247883.0
18  2013         42701765.0
19  2014         41365442.0
20  2015         42454507.0
21  2016         42500923.0
22  2017         42906645.0
23  2018         43477236.0
24  2019         42227183.0
25  2020         39140244.0


In [ ]:
query_percentage_contribution = """
SELECT Year, "Energy Carriers",
       SUM(Energy_Usage) * 100.0 /
       (SELECT SUM(Energy_Usage)
        FROM Energy_Use_Carriers_Long
        WHERE Year = e.Year AND "Energy Carriers" = 'Total') AS Contribution_Percentage
FROM Energy_Use_Carriers_Long e
WHERE "Energy Carriers" != 'Total'
GROUP BY Year, "Energy Carriers"
ORDER BY Year, "Energy Carriers";
"""
result_contribution = pd.read_sql_query(query_percentage_contribution, conn)
print(result_contribution)


     Year                               Energy Carriers  \
0    1995                         Aviation turbine fuel   
1    1995  Brown coal (lignite) and brown coal products   
2    1995                                     Crude oil   
3    1995                                  Diesel fuels   
4    1995          Electricity and other energy sources   
..    ...                                           ...   
307  2020                                   Heating oil   
308  2020                                Heavy fuel oil   
309  2020                    Other mineral oil products   
310  2020                                        Petrol   
311  2020                            Renewable energies   

     Contribution_Percentage  
0                   1.652473  
1                   8.974304  
2                  20.920662  
3                   5.131529  
4                  17.736226  
..                       ...  
307                 3.394307  
308                 2.661131  
309         

In [ ]:
result_carriers.to_csv('total_carriers.csv', index=False)
result_total_year.to_csv('total_year.csv', index=False)
result_contribution.to_csv('contribution_percentage.csv', index=False)


In [ ]:
# Export total energy consumption by carrier
result_carriers.to_csv('total_carriers.csv', index=False)

# Export total energy consumption by year
result_total_year.to_csv('total_year.csv', index=False)

# Export contribution percentages
result_contribution.to_csv('contribution_percentage.csv', index=False)

print("Results exported as CSV files.")


Results exported as CSV files.
